In [1]:
!mkdir .kaggle

In [2]:
import json
token = {"username": "myseolumic", "key": "0085cfc466ca2506e25ef86587c58eae"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [4]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 ~/.kaggle/kaggle.json

- path is now set to: {/content}


In [5]:
!kaggle competitions download -c competitive-data-science-predict-future-sales

  0% 0.00/468k [00:00<?, ?B/s]
100% 468k/468k [00:00<00:00, 70.3MB/s]
  0% 0.00/1.02M [00:00<?, ?B/s]
100% 1.02M/1.02M [00:00<00:00, 67.6MB/s]
 68% 9.00M/13.3M [00:00<00:00, 35.1MB/s]
100% 13.3M/13.3M [00:00<00:00, 44.2MB/s]
  0% 0.00/368k [00:00<?, ?B/s]
100% 368k/368k [00:00<00:00, 111MB/s]
  0% 0.00/3.49k [00:00<?, ?B/s]
100% 3.49k/3.49k [00:00<00:00, 3.81MB/s]
  0% 0.00/2.91k [00:00<?, ?B/s]
100% 2.91k/2.91k [00:00<00:00, 2.77MB/s]


In [6]:
!unzip sample_data/competitive-data-science-predict-future-sales/sales_train.csv.zip
!unzip sample_data/competitive-data-science-predict-future-sales/test.csv.zip
!unzip sample_data/competitive-data-science-predict-future-sales/sample_submission.csv.zip
!unzip sample_data/competitive-data-science-predict-future-sales/items.csv.zip

Archive:  sample_data/competitive-data-science-predict-future-sales/sales_train.csv.zip
  inflating: sales_train.csv         
Archive:  sample_data/competitive-data-science-predict-future-sales/test.csv.zip
  inflating: test.csv                
Archive:  sample_data/competitive-data-science-predict-future-sales/sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  sample_data/competitive-data-science-predict-future-sales/items.csv.zip
  inflating: items.csv               


In [7]:
!mv sample_data/competitive-data-science-predict-future-sales/item_categories.csv item_categories.csv
!mv sample_data/competitive-data-science-predict-future-sales/shops.csv shops.csv

In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

data_root = "../sample_data/competitive-data-science-predict-future-sales/"

train = pd.read_csv('sales_train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')
items = pd.read_csv('items.csv')
cats = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [57]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [48]:
#We are asking you to predict total sales for every product and store in the next month.
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
train = train.set_index('item_id').join(items.set_index('item_id')).drop('item_name', axis=1).reset_index()
train['month'] = train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%m'))
train['year'] = train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%Y'))
train = train.query('item_cnt_day >= 0 and item_cnt_day <= 125 and item_price < 75000')
#train = train.drop('date', axis=1)
#train = train.drop('date_block_num', axis=1)
train.head()

,item_id,date,date_block_num,shop_id,item_price,item_cnt_day,item_category_id,month,year
0,0,01.09.2014,20,54,58.0,1.0,40,09,2014
1,1,04.04.2014,15,55,4490.0,1.0,76,04,2014
2,1,02.04.2014,15,55,4490.0,1.0,76,04,2014
3,1,06.07.2014,18,55,4490.0,1.0,76,07,2014
4,1,04.08.2014,19,55,4490.0,1.0,76,08,2014


In [45]:
train_m=train.groupby(["date_block_num","shop_id","item_id"])["date_block_num","date","item_price","item_cnt_day"].agg(
    {"date_block_num":'mean',"date":["min",'max'],"item_price":"mean","item_cnt_day":"sum"})

train_m.head(5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


date_block_num  ... item_cnt_day
                                         mean  ...          sum
date_block_num shop_id item_id                 ...             
0              0       32                   0  ...          6.0
                       33                   0  ...          3.0
                       35                   0  ...          1.0
                       43                   0  ...          1.0
                       51                   0  ...          2.0

[5 rows x 5 columns]

In [49]:
train_flat = train_m.item_cnt_day.apply(list).reset_index()
train_flat = pd.merge(test,train_flat,on = ['item_id','shop_id'],how = 'left')
train_flat.fillna(0,inplace = True)
train_flat.drop(['shop_id','item_id'],inplace = True, axis = 1)
train_flat.head(20)

,ID,date_block_num,sum
0,0,20.0,1.0
1,0,22.0,1.0
2,0,23.0,2.0
3,0,24.0,2.0
4,0,28.0,1.0
5,0,29.0,1.0
6,0,30.0,1.0
7,0,31.0,3.0
8,0,32.0,1.0
9,1,0.0,0.0


In [52]:
pivoted_train = train_flat.pivot_table(index='ID', columns='date_block_num',fill_value = 0,aggfunc='sum' )
pivoted_train.head(2)

sum                           ...                              
date_block_num 0.0  1.0  2.0  3.0  4.0  5.0   ... 28.0 29.0 30.0 31.0 32.0 33.0
ID                                            ...                              
0                 0    0    0    0    0    0  ...    1    1    1    3    1    0
1                 0    0    0    0    0    0  ...    0    0    0    0    0    0

[2 rows x 34 columns]

In [61]:
X_train = sales_data_flat[["ID", "date_block_num"]]
y_train = sales_data_flat[["sum"]]

X_train_p = np.expand_dims(pivoted_train.values[:,:-1],axis = 2)
y_train_p = pivoted_train.values[:,-1:]
X_test_p = np.expand_dims(pivoted_train.values[:,1:],axis = 2)

X_train.shape

(702955, 2)

Neural Network:

In [92]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
from keras.models import load_model, Model

lstm_model = Sequential()
lstm_model.add(LSTM(units = 64,input_shape = (33,1)))
lstm_model.add(Dropout(0.4))
lstm_model.add(Dense(1))

lstm_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
lstm_model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 64)                16896     
_________________________________________________________________
dropout_11 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 65        
Total params: 16,961
Trainable params: 16,961
Non-trainable params: 0
_________________________________________________________________


In [88]:
from keras.models import Sequential
from keras.layers import Input,Dense,Dropout
from keras.models import load_model, Model

other_model = Sequential()
other_model.add(Input((33,1)))
other_model.add(Dense(64, activation="sigmoid"))
other_model.add(Dropout(0.5))
other_model.add(Dense(32, activation="relu"))
other_model.add(Dense(1))

other_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
other_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 33, 64)            128       
_________________________________________________________________
dropout_9 (Dropout)          (None, 33, 64)            0         
_________________________________________________________________
dense_15 (Dense)             (None, 33, 32)            2080      
_________________________________________________________________
dense_16 (Dense)             (None, 33, 1)             33        
Total params: 2,241
Trainable params: 2,241
Non-trainable params: 0
_________________________________________________________________


In [93]:
lstm_model.fit(X_train_p,y_train_p,batch_size = 4096,epochs = 20)

Epoch 1/20
53/53 [==============================] - 34s 638ms/step - loss: 30.7109 - mean_squared_error: 30.7109
Epoch 2/20
53/53 [==============================] - 33s 630ms/step - loss: 30.3383 - mean_squared_error: 30.3383
Epoch 3/20
53/53 [==============================] - 33s 627ms/step - loss: 30.0751 - mean_squared_error: 30.0751
Epoch 4/20
53/53 [==============================] - 33s 627ms/step - loss: 29.9870 - mean_squared_error: 29.9870
Epoch 5/20
53/53 [==============================] - 33s 626ms/step - loss: 29.8561 - mean_squared_error: 29.8561
Epoch 6/20
53/53 [==============================] - 33s 622ms/step - loss: 29.7886 - mean_squared_error: 29.7886
Epoch 7/20
53/53 [==============================] - 33s 626ms/step - loss: 29.6750 - mean_squared_error: 29.6750
Epoch 8/20
53/53 [==============================] - 33s 630ms/step - loss: 29.6001 - mean_squared_error: 29.6001
Epoch 9/20
53/53 [==============================] - 34s 647ms/step - loss: 29.4868 - mean_square

In [89]:
other_model.fit(X_train_p,y_train_p,batch_size = 4096,epochs = 20)

Epoch 1/20
53/53 [==============================] - 10s 182ms/step - loss: 31.0082 - mean_squared_error: 31.0082
Epoch 2/20
53/53 [==============================] - 10s 182ms/step - loss: 30.8284 - mean_squared_error: 30.8284
Epoch 3/20
53/53 [==============================] - 10s 181ms/step - loss: 30.7593 - mean_squared_error: 30.7593
Epoch 4/20
53/53 [==============================] - 10s 184ms/step - loss: 30.7181 - mean_squared_error: 30.7181
Epoch 5/20
53/53 [==============================] - 10s 183ms/step - loss: 30.6666 - mean_squared_error: 30.6666
Epoch 6/20
53/53 [==============================] - 10s 181ms/step - loss: 30.6132 - mean_squared_error: 30.6132
Epoch 7/20
53/53 [==============================] - 10s 182ms/step - loss: 30.5556 - mean_squared_error: 30.5556
Epoch 8/20
53/53 [==============================] - 10s 181ms/step - loss: 30.4961 - mean_squared_error: 30.4961
Epoch 9/20
53/53 [==============================] - 10s 182ms/step - loss: 30.4350 - mean_square

In [80]:
submission_output = other_model.predict(X_test_p)
#submission_output = sales_model.predict(X_test_p)
submission = pd.DataFrame({'ID':test['ID'],'item_cnt_month':submission_output.ravel()})

submission.to_csv('submission_stacked.csv',index = False)
submission.head()

,ID,item_cnt_month
0,0,0.454383
1,1,0.175963
2,2,0.620022
3,3,0.212913
4,4,0.175963


In [81]:
!kaggle competitions submit -c competitive-data-science-predict-future-sales -f submission_stacked.csv -m "Test submission 2"

100% 3.44M/3.44M [00:00<00:00, 13.6MB/s]
Successfully submitted to Predict Future Sales